In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras_squeezenet import SqueezeNet
from keras.optimizers import Adam
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.models import Sequential

In [2]:
IMG_SAVE_PATH = 'images'

CLASS_MAP = {
    "rock": 0,
    "paper": 1,
    "scissors": 2,
    "none": 3
}

NUM_CLASSES = len(CLASS_MAP)

def mapper(val):
    return CLASS_MAP[val]

def get_model():
    model = Sequential([
        SqueezeNet(input_shape=(227, 227, 3), include_top=False),
        Dropout(0.5),
        Convolution2D(NUM_CLASSES, (1, 1), padding='valid'),
        Activation('relu'),
        GlobalAveragePooling2D(),
        Flatten(),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
        if item.startswith("."):
            continue
        img = cv2.imread(os.path.join(path, item))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (227, 227))
        dataset.append([img, directory])

data, labels = zip(*dataset)
labels = list(map(mapper, labels))


labels = to_categorical(labels)


In [3]:
model = get_model()
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(np.array(data), np.array(labels), epochs=15)


75/75 ━━━━━━━━━━━━━━━━━━━━ 62s 705ms/step - accuracy: 0.6680 - loss: 1.1068


In [4]:
REV_CLASS_MAP = {
    0: "rock",
    1: "paper",
    2: "scissors",
    3: "none"
}

def mapper(val):
    return REV_CLASS_MAP[val]

def calculate_winner(move1, move2):
    if move1 == move2:
        return 0

    if move1 == "rock":
        if move2 == "scissors":
            return 1
        if move2 == "paper":
            return 2

    if move1 == "paper":
        if move2 == "rock":
            return 1
        if move2 == "scissors":
            return 2

    if move1 == "scissors":
        if move2 == "paper":
            return 1
        if move2 == "rock":
            return 2

cap = cv2.VideoCapture(0)

score1 = 0
score2 = 0

move1_scored = False
move2_scored = False

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame , (1100,700))
    if not ret and frame:
        continue
    
    cv2.rectangle(frame, (100, 100), (500, 500), (0, 0, 255), 2)
    cv2.rectangle(frame, (600, 100), (1000, 500), (0, 0, 255), 2)

    roi = frame[100:500, 100:500]
    img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (227, 227))
    pred = model.predict(np.array([img]))
    move_code = np.argmax(pred[0])
    user_move_name = mapper(move_code)
    
    roi2 = frame[100:500,600:1000]
    img2 = cv2.cvtColor(roi2, cv2.COLOR_BGR2RGB)
    img2 = cv2.resize(img2, (227, 227))
    pred2 = model.predict(np.array([img2]))
    move_code2 = np.argmax(pred2[0])
    user_move_name2 = mapper(move_code2)

    chck = calculate_winner(user_move_name, user_move_name2)

    if chck == 1 and not move1_scored:
        score1 += 1
        move1_scored = True
        move2_scored = False
    elif chck == 2 and not move2_scored:
        score2 += 1
        move1_scored = False
        move2_scored = True
    elif chck == 0:
        move1_scored = False
        move2_scored = False
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "User1's Move: " + user_move_name,
                (50, 50), font, 1.2, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "User2's Move: " + user_move_name2,
                (550, 50), font, 1.2, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, str(score1) + " - " + str(score2),
                (400, 600), font, 2, (255, 0, 0), 4, cv2.LINE_AA)

    cv2.imshow("Rock Paper Scissors", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━